In [1]:
# MongoDB dependencies & setup

import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# BeautifulSoup dependecies

from bs4 import BeautifulSoup as bs

# request and json dependencies
import requests
import json
from pprint import pprint

import pandas as pd

from flask import Flask, render_template, redirect

from splinter import Browser


# Get Mars News

In [2]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

mars_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_news_url)

soup = bs(browser.html, 'html.parser')

news_title = soup.find('div', class_='content_title').text
news_overview= soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_overview)


Curiosity Tastes First Sample in 'Clay-Bearing Unit'
This new region on Mars might reveal more about the role of water on Mount Sharp.


# Get JPL Featured Mars Image

In [3]:
browser = Browser('chrome', **executable_path, headless=True)

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

browser.click_link_by_partial_text('FULL IMAGE')

soup = bs(browser.html, 'html.parser')
mars_image = soup.find('article')


mars_image_url = "https://www.jpl.nasa.gov/spaceimages" + mars_image['style'].split('spaceimages')[1].split("');")[0]
print(mars_image_url)


https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18614-1920x1200.jpg


# Get Mars Weather


In [4]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

html = requests.get(mars_weather_url).text

soup = bs(html, 'html.parser')

mars_weather = soup.find('div', class_='js-tweet-text-container').p.text
print(mars_weather)

InSight sol 137 (2019-04-16) low -97.0ºC (-142.7ºF) high -15.9ºC (3.4ºF)
winds from the SW at 4.3 m/s (9.7 mph) gusting to 12.4 m/s (27.7 mph)
pressure at 7.30 hPapic.twitter.com/jhaMb7q9gy


# Get Mars Facts

In [111]:
mars_facts_url = 'https://space-facts.com/mars/'

html = requests.get(mars_facts_url).text

soup = bs(html, 'html.parser')

table = soup.find('table')

mars_facts_df = pd.read_html(str(table))[0]
mars_facts_df.columns = ['Description', 'Value']
mars_facts_df.set_index(['Description', 'Value'], inplace=True)

mars_facts_df

,
Description,Value
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [112]:
mars_facts_df.to_html('mars_facts_table.html') #### Can be deleted once table is cleaned up

# Get Mars Hemisphere Images

In [100]:
mars_hemispere_img_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser = Browser('chrome', **executable_path, headless=True)
browser.visit(mars_hemispere_img_url)

soup = bs(browser.html, 'html.parser')

image_links = soup.find_all('div', class_='item')

hemisphere_image_urls = []
links_to_follow = []

for link in image_links:
    link_text = link.h3.text
    browser.click_link_by_partial_text(link_text)
    soup = bs(browser.html, 'html.parser')
    image_link = 'https://astrogeology.usgs.gov' + soup.find('img', class_='wide-image')['src']
    hemisphere_image_urls.append({'title' : link_text, 'img_url' : image_link})
    browser.back()
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
